<a href="https://colab.research.google.com/github/James-H05/SSI_2021/blob/main/ee_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import earth engine library - should be installed already in colab
import ee

In [ ]:
FRANCE = ee.Geometry.Polygon(
        [[[-4.979612038107639, 48.54231434082373],
          [-4.408322975607639, 47.883445781121715],
          [-2.584592506857639, 47.54341404042898],
          [-1.6837136006076392, 46.570445564584084],
          [-1.0025612568576392, 46.08491916839342],
          [-1.1783425068576392, 45.84053946233962],
          [-1.3980690693576392, 43.70389064764542],
          [-1.9034401631076392, 43.36938872863792],
          [-0.8926979756076392, 42.92050085551135],
          [0.2498801493923608, 42.69482128306705],
          [0.6234153056423608, 42.72711162875077],
          [0.6453879618923608, 42.88831133060359],
          [1.3622458720486108, 42.69482128306705],
          [1.4418967509548608, 42.60997920653104],
          [1.4940818095486108, 42.646354268237225],
          [1.7110617900173608, 42.616043192928984],
          [1.7962058329861108, 42.57965041843404],
          [1.7165549540798608, 42.502751435967475],
          [1.9719870829861108, 42.43792094611423],
          [1.9692405009548608, 42.36490639024963],
          [2.048891379861111, 42.35881800880227],
          [2.150514915017361, 42.419675272922554],
          [2.356508567361111, 42.42373032592709],
          [2.463625266579861, 42.3506992491502],
          [2.606447532204861, 42.3506992491502],
          [2.675112082986111, 42.354758760077985],
          [2.749269797829861, 42.42170283220727],
          [2.905824973611111, 42.46426643105712],
          [3.125551536111111, 42.43183964514928],
          [3.169496848611111, 42.439947915299996],
          [3.081606223611111, 42.541212766011924],
          [3.032167747048611, 42.73114674075607],
          [3.065126731423611, 43.00692691280765],
          [3.251894309548611, 43.225462834769274],
          [3.490846946267361, 43.277475371281234],
          [3.683107688454861, 43.39733475595107],
          [3.897341086892361, 43.52890677903396],
          [4.067629172829861, 43.57270048277342],
          [4.171999290017361, 43.477109559319764],
          [4.435671165017361, 43.46116301264572],
          [4.655397727517361, 43.42526790051358],
          [5.039919211892361, 43.329443486257766],
          [5.342043235329861, 43.30146617524505],
          [5.413454368142361, 43.20144206763377],
          [5.825441672829861, 43.085208328001976],
          [6.177004172829861, 43.04106186275166],
          [6.588991477517361, 43.15737942251023],
          [6.940553977517361, 43.48508125494492],
          [7.193239524392361, 43.62043913843813],
          [7.517336204079861, 43.78722614932898],
          [7.719023416903665, 44.07119065529815],
          [7.554228495028665, 44.1894734328751],
          [7.213652323153665, 44.17371608267171],
          [6.840117166903665, 44.43317377576282],
          [7.037871073153665, 44.714910590846905],
          [6.928007791903665, 44.91753827408833],
          [6.631376932528665, 45.08843632956073],
          [6.906035135653665, 45.19692420700843],
          [7.125761698153665, 45.34382755451029],
          [6.971953104403665, 45.567315419391896],
          [6.840117166903665, 45.8052368713228],
          [7.026884745028665, 45.93528033530401],
          [6.884062479403665, 46.11835317193932],
          [6.796171854403665, 46.42970023969105],
          [6.279814432528665, 46.40697872132886],
          [6.158964823153665, 46.17163521345079],
          [5.983183573153665, 46.110737241817745],
          [6.115019510653665, 46.308409392552896],
          [6.071074198153665, 46.45241229014722],
          [6.235869120028665, 46.66393493530407],
          [6.532499979403665, 46.94968284599772],
          [6.884062479403665, 47.189135665465145],
          [6.960966776278665, 47.43494503076834],
          [7.224638651278665, 47.479514827485175],
          [7.488310526278665, 47.48693945404701],
          [7.598173807528665, 47.575953131514176],
          [7.565214823153665, 47.98935362060113],
          [7.631132791903665, 48.28262808413062],
          [7.916777323153665, 48.683147202792014],
          [8.191435526278665, 48.97965992303464],
          [7.719023416903665, 49.07330988637416],
          [7.444365213778665, 49.19550935478718],
          [7.136748026278665, 49.13085304341639],
          [6.938994120028665, 49.21704270073452],
          [6.752226541903665, 49.15241452716332],
          [6.532499979403665, 49.41756854886525],
          [6.323759745028665, 49.49612325997176],
          [5.972197245028665, 49.46757247831247],
          [5.785429666903665, 49.54604707577752],
          [5.455839823153665, 49.50325835439568],
          [5.269072245028665, 49.659967305478375],
          [4.873564432528665, 49.8232615114039],
          [4.785673807528665, 49.95774237797015],
          [4.862578104403665, 50.19745770652949],
          [4.730742166903665, 50.148202513566154],
          [4.653837870028665, 50.049539770048774],
          [4.401152323153665, 49.95774237797015],
          [4.137480448153665, 49.978941951446636],
          [4.159453104403665, 50.16228060684049],
          [4.137480448153665, 50.2887969667443],
          [3.7090136512786653, 50.3589388594789],
          [3.5991503700286653, 50.51987253786077],
          [3.3244921669036653, 50.491923373516464],
          [3.2366015419036653, 50.70809690700861],
          [2.9839159950286653, 50.78456506882797],
          [2.8410937294036653, 50.722009499332735],
          [2.5334765419036653, 51.116784054053305],
          [1.5996386512786653, 50.88863862761188],
          [1.3469531044036653, 50.141161912443756],
          [0.050566385653665336, 49.759429227432015],
          [-0.32296877059633466, 49.40327234526095],
          [-1.0590527549713347, 49.46757247831247],
          [-1.2897656455963347, 49.787809517526725],
          [-2.0807812705963347, 49.766525858045185],
          [-1.5973828330963347, 48.719401257263286],
          [-2.8498242393463347, 48.81353961070384],
          [-3.9045117393463347, 48.96523659108451]]]
)